# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [64]:
! jupyter nbextension enable --py --sys-prefix widgetsnbextension

Enabling notebook extension jupyter-js-widgets/extension...
      - Validating: ok


In [65]:
!jupyter nbextension enable --py --sys-prefix gmaps

Enabling notebook extension jupyter-gmaps/extension...
      - Validating: ok


In [81]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [82]:
df_weather = pd.read_csv("../output_data/cities.csv")
df_weather.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Butaritari,3.0707,172.7902,82.83,79,40,15.23,KI,1621130074
1,Ahipara,-35.1667,173.1667,67.15,68,2,10.65,NZ,1621130091
2,Rikitea,-23.1203,-134.9692,74.55,67,100,17.92,PF,1621130203
3,Yellowknife,62.4560,-114.3525,39.07,65,75,12.66,CA,1621130203
4,Sesimbra,38.4445,-9.1015,63.72,77,75,6.91,PT,1621130204


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [83]:
gmaps.configure(api_key=g_key)

locations = df_weather[["Lat", "Lng"]]

humidity = df_weather["Humidity"]

In [84]:
fig = gmaps.figure(center=(46.0, -5.0), zoom_level=2)
max_intensity = np.max(humidity)

heat_layer = gmaps.heatmap_layer(locations, weights = humidity, dissipating=False, max_intensity=100, point_radius=3)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

In [85]:
max_intensity

100

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [86]:
df_new_weather = df_weather[(df_weather['Max Temp'] >= 75) & (df_weather['Max Temp'] <= 90)]
df_new_weather = df_new_weather[df_new_weather['Wind Speed'] <= 10]
df_new_weather = df_new_weather[df_new_weather['Cloudiness'] <= 0]
df_new_weather = df_new_weather[df_new_weather['Humidity'] <= 70]
hotel_df = df_new_weather

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [87]:
hotel_df['Hotel Name'] = ""
hotel_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
49,Buraidah,26.3260,43.9750,84.20,20,0,6.91,SA,1621130223,
214,Riyadh,24.6877,46.7219,84.67,16,0,6.06,SA,1621130015,
308,Tomatlán,19.9333,-105.2500,81.07,58,0,3.89,MX,1621130336,
360,Gambiran,-6.7988,111.4951,83.46,66,0,6.51,ID,1621130357,
392,Sawākin,19.1059,37.3321,80.71,41,0,2.24,SD,1621130369,
473,Grand Gaube,-20.0064,57.6608,75.99,68,0,5.99,MU,1621130400,


In [91]:
for index, row in hotel_df.iterrows():

    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json?"

    params = {
    "type":"hotel",
    "radius": 5000,
    "key":g_key
    }

    lat = row['Lat']
    lng = row['Lng']
    city_name = row["City"]

    params['location'] = f"{lat}, {lng}"

    hotel_data = requests.get(base_url, params=params).json()

    results = hotel_data['results']

    try:

        hotel_df.loc[index, "Hotel Name"] = results[0]["name"]
        
    except IndexError:
        
        hotel_df.loc[index, "Hotel Name"] = "NaN"
        
hotel_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
49,Buraidah,26.3260,43.9750,84.20,20,0,6.91,SA,1621130223,Buraydah
214,Riyadh,24.6877,46.7219,84.67,16,0,6.06,SA,1621130015,Riyadh
308,Tomatlán,19.9333,-105.2500,81.07,58,0,3.89,MX,1621130336,Tomatlán
360,Gambiran,-6.7988,111.4951,83.46,66,0,6.51,ID,1621130357,Pamotan
392,Sawākin,19.1059,37.3321,80.71,41,0,2.24,SD,1621130369,Suakin
473,Grand Gaube,-20.0064,57.6608,75.99,68,0,5.99,MU,1621130400,Goodlands


In [89]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [90]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(markers)
# Display figure
fig

Figure(layout=FigureLayout(height='420px'))